In [ ]:
import os
from itertools import permutations

In [190]:
template = """
FROM prolocutor/docker-python-sqlite:3.12
#FROM 229931fb6377
RUN cd /tmp/sqlite-autoconf-3120200 \                                                                    
        && ./configure --prefix=/usr --disable-static \                                                  
                CFLAGS="-g {{ flag -}}" \
        && make \                                                                                        
        && make install \                                                                                
        && rm -r /tmp/*                                                                                  

CMD ["python3"]

"""
root = "./sqlite3.12/"
with open(root+"Dockerfile.jinja2","w") as f:
    f.write(template)

In [191]:
tree = {"sqlite3.12":"./sqlite3.12"}

trunk = {"Os":"-Os",
         "O2":"-O2",
        "debug":"-DSQLITE_DEBUG"}

available_flags = {"fts5":"-DSQLITE_ENABLE_FTS5",
                   "rtree":"-DSQLITE_ENABLE_RTREE",
                   "column-metadata":"-DSQLITE_ENABLE_COLUMN_METADATA",
                   "json1":"-DSQLITE_ENABLE_JSON1"}

dockerfile_dir = []
for root in tree.values():
    for p in permutations(available_flags):
        for item in trunk:
            d = os.path.join(root,item,"/".join(p))
            try: os.makedirs(d)
            except: pass
            dockerfile_dir.append(d)
dockerfile_dir

['./sqlite3.12/Os/column-metadata/fts5/json1/rtree',
 './sqlite3.12/O2/column-metadata/fts5/json1/rtree',
 './sqlite3.12/debug/column-metadata/fts5/json1/rtree',
 './sqlite3.12/Os/column-metadata/fts5/rtree/json1',
 './sqlite3.12/O2/column-metadata/fts5/rtree/json1',
 './sqlite3.12/debug/column-metadata/fts5/rtree/json1',
 './sqlite3.12/Os/column-metadata/json1/fts5/rtree',
 './sqlite3.12/O2/column-metadata/json1/fts5/rtree',
 './sqlite3.12/debug/column-metadata/json1/fts5/rtree',
 './sqlite3.12/Os/column-metadata/json1/rtree/fts5',
 './sqlite3.12/O2/column-metadata/json1/rtree/fts5',
 './sqlite3.12/debug/column-metadata/json1/rtree/fts5',
 './sqlite3.12/Os/column-metadata/rtree/fts5/json1',
 './sqlite3.12/O2/column-metadata/rtree/fts5/json1',
 './sqlite3.12/debug/column-metadata/rtree/fts5/json1',
 './sqlite3.12/Os/column-metadata/rtree/json1/fts5',
 './sqlite3.12/O2/column-metadata/rtree/json1/fts5',
 './sqlite3.12/debug/column-metadata/rtree/json1/fts5',
 './sqlite3.12/Os/fts5/colum

In [192]:

flag_combinations = []

for L in range(0, len(available_flags)+1):
    for subset in itertools.combinations(available_flags, L):
        for k,v in trunk.items():
            flag_combinations.append(sorted(subset+(k,)))
            
flag_combinations

[['Os'],
 ['O2'],
 ['debug'],
 ['Os', 'column-metadata'],
 ['O2', 'column-metadata'],
 ['column-metadata', 'debug'],
 ['Os', 'fts5'],
 ['O2', 'fts5'],
 ['debug', 'fts5'],
 ['Os', 'json1'],
 ['O2', 'json1'],
 ['debug', 'json1'],
 ['Os', 'rtree'],
 ['O2', 'rtree'],
 ['debug', 'rtree'],
 ['Os', 'column-metadata', 'fts5'],
 ['O2', 'column-metadata', 'fts5'],
 ['column-metadata', 'debug', 'fts5'],
 ['Os', 'column-metadata', 'json1'],
 ['O2', 'column-metadata', 'json1'],
 ['column-metadata', 'debug', 'json1'],
 ['Os', 'column-metadata', 'rtree'],
 ['O2', 'column-metadata', 'rtree'],
 ['column-metadata', 'debug', 'rtree'],
 ['Os', 'fts5', 'json1'],
 ['O2', 'fts5', 'json1'],
 ['debug', 'fts5', 'json1'],
 ['Os', 'fts5', 'rtree'],
 ['O2', 'fts5', 'rtree'],
 ['debug', 'fts5', 'rtree'],
 ['Os', 'json1', 'rtree'],
 ['O2', 'json1', 'rtree'],
 ['debug', 'json1', 'rtree'],
 ['Os', 'column-metadata', 'fts5', 'json1'],
 ['O2', 'column-metadata', 'fts5', 'json1'],
 ['column-metadata', 'debug', 'fts5', 'j

In [193]:
all_flags = available_flags.update(trunk)
print(available_flags)

{'O2': '-O2', 'debug': '-DSQLITE_DEBUG', 'rtree': '-DSQLITE_ENABLE_RTREE', 'column-metadata': '-DSQLITE_ENABLE_COLUMN_METADATA', 'fts5': '-DSQLITE_ENABLE_FTS5', 'Os': '-Os', 'json1': '-DSQLITE_ENABLE_JSON1'}


In [194]:
from jinja2 import Template, Environment, FileSystemLoader
import collections

#Everything is a template
obreros_env = Environment(loader=FileSystemLoader('.'))

for k,v in tree.items():
    for root, subdirs, files in os.walk(v):
        root_flags = root.replace(v,'').split('/')[1:]

        for flags in flag_combinations:
            if collections.Counter(root_flags) == collections.Counter(flags):
                flag = " ".join([available_flags[flag] for flag in flags])
                print(flag)
                obreros_template = obreros_env.get_template('Dockerfile.jinja2').render(flag=flag)
                Dockerfile = root+"/"+"Dockerfile"
                print(Dockerfile)
                with open(Dockerfile, "w") as f:
                    f.write(obreros_template)

-O2
./sqlite3.12/O2/Dockerfile
-O2 -DSQLITE_ENABLE_RTREE
./sqlite3.12/O2/rtree/Dockerfile
-O2 -DSQLITE_ENABLE_JSON1 -DSQLITE_ENABLE_RTREE
./sqlite3.12/O2/rtree/json1/Dockerfile
-O2 -DSQLITE_ENABLE_COLUMN_METADATA -DSQLITE_ENABLE_JSON1 -DSQLITE_ENABLE_RTREE
./sqlite3.12/O2/rtree/json1/column-metadata/Dockerfile
-O2 -DSQLITE_ENABLE_COLUMN_METADATA -DSQLITE_ENABLE_FTS5 -DSQLITE_ENABLE_JSON1 -DSQLITE_ENABLE_RTREE
./sqlite3.12/O2/rtree/json1/column-metadata/fts5/Dockerfile
-O2 -DSQLITE_ENABLE_FTS5 -DSQLITE_ENABLE_JSON1 -DSQLITE_ENABLE_RTREE
./sqlite3.12/O2/rtree/json1/fts5/Dockerfile
-O2 -DSQLITE_ENABLE_COLUMN_METADATA -DSQLITE_ENABLE_FTS5 -DSQLITE_ENABLE_JSON1 -DSQLITE_ENABLE_RTREE
./sqlite3.12/O2/rtree/json1/fts5/column-metadata/Dockerfile
-O2 -DSQLITE_ENABLE_COLUMN_METADATA -DSQLITE_ENABLE_RTREE
./sqlite3.12/O2/rtree/column-metadata/Dockerfile
-O2 -DSQLITE_ENABLE_COLUMN_METADATA -DSQLITE_ENABLE_JSON1 -DSQLITE_ENABLE_RTREE
./sqlite3.12/O2/rtree/column-metadata/json1/Dockerfile
-O2 -DSQLIT

In [201]:
! wget https://github.com/disarticulate/docker-python-sqlite/tree/master/sqlite3.12/Dockerfile

--2016-05-09 22:01:45--  https://github.com/disarticulate/docker-python-sqlite/tree/master/sqlite3.12/Dockerfile
Resolving github.com (github.com)... 192.30.252.128
Connecting to github.com (github.com)|192.30.252.128|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/disarticulate/docker-python-sqlite/blob/master/sqlite3.12/Dockerfile [following]
--2016-05-09 22:01:46--  https://github.com/disarticulate/docker-python-sqlite/blob/master/sqlite3.12/Dockerfile
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘Dockerfile.1’

Dockerfile.1            [ <=>                ]  47.02K  --.-KB/s    in 0.1s    

2016-05-09 22:01:46 (322 KB/s) - ‘Dockerfile.1’ saved [48151]

